## DO NOT RUN THE CELLS. CODE is included here for you to refer to for your future projects.

````py
# Max Voting 

from sklearn.ensemble import VotingClassifier

model1 = LogisticRegression(random_state=1)

model2 = tree.DecisionTreeClassifier(random_state=1)

model = VotingClassifier(estimators=[('lr', model1), ('dt', model2)], voting='hard')   #voting between the two models to get the predictions

model.fit(x_train,y_train)

model.score(x_test,y_test)
````

````py
# Averaging

model1 = tree.DecisionTreeClassifier()

model2 = KNeighborsClassifier()

model3= LogisticRegression()

model1.fit(x_train,y_train)

model2.fit(x_train,y_train)

model3.fit(x_train,y_train)

pred1=model1.predict_proba(x_test)

pred2=model2.predict_proba(x_test)

pred3=model3.predict_proba(x_test)

finalpred=(pred1+pred2+pred3)/3    # average over the three predictions
````

````py
# Weighted Average

model1 = tree.DecisionTreeClassifier()

model2 = KNeighborsClassifier()

model3= LogisticRegression()

model1.fit(x_train,y_train)

model2.fit(x_train,y_train)

model3.fit(x_train,y_train)

pred1=model1.predict_proba(x_test)

pred2=model2.predict_proba(x_test)

pred3=model3.predict_proba(x_test)

finalpred=(pred1*0.3+pred2*0.3+pred3*0.4) # weigthed average
````

````py
# Stacking 

def Stacking(model,train,y,test,n_fold):
   folds=StratifiedKFold(n_splits=n_fold,random_state=1)   #creating folds for the stacking 
   test_pred=np.empty((test.shape[0],1),float)             #array to put tests in
   train_pred=np.empty((0,1),float)                        #array to put trains in
   for train_indices,val_indices in folds.split(train,y.values):
      x_train,x_val=train.iloc[train_indices],train.iloc[val_indices]  #getting train and validations
      y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]          #by using the random fold distibution

      model.fit(X=x_train,y=y_train)                       #fitting the model on these
      train_pred=np.append(train_pred,model.predict(x_val))#appending the results to the arrays above
      test_pred=np.append(test_pred,model.predict(test))   #note that the predict array will hold the results 
                                                           #over every single step
    return test_pred.reshape(-1,1),train_pred
````

In [ ]:
# CReating 2 base models

model1 = tree.DecisionTreeClassifier(random_state=1)

test_pred1 ,train_pred1=Stacking(model=model1,n_fold=10, train=x_train,test=x_test,y=y_train)
#Stacking on Dec. Tree with the above function

train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)



model2 = KNeighborsClassifier()

test_pred2 ,train_pred2=Stacking(model=model2,n_fold=10,train=x_train,test=x_test,y=y_train)

train_pred2=pd.DataFrame(train_pred2)
test_pred2=pd.DataFrame(test_pred2)


In [ ]:
Create a third model, logistic regression, on the predictions of the decision tree and knn models.

#basically a stack by using the above prediction in a logistic regression

df = pd.concat([train_pred1, train_pred2], axis=1)
df_test = pd.concat([test_pred1, test_pred2], axis=1)

model = LogisticRegression(random_state=1)
model.fit(df,y_train)
model.score(df_test, y_test)

In order to simplify the above explanation, the stacking model we have created has only two levels. The decision tree and knn models are built at level zero, while a logistic regression model is built at level one. Feel free to create multiple levels in a stacking model.

 